##**Primer ejercicio Spark RRS**
20 de abril 2020


Primer paso: **Instalación de Spark y descomprimir**

In [ ]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.5.1/spark-3.5.1-bin-hadoop3.tgz
!tar xvf spark-3.5.1-bin-hadoop3.tgz

In [ ]:
!ls

deporte.csv	     drive	 modelo_relacional.jpg	spark-3.5.1-bin-hadoop3
deportista2.csv      evento.csv  paises.csv		spark-3.5.1-bin-hadoop3.tgz
deportista.csv	     gdrive	 resultados.csv		spark-3.5.1-bin-hadoop3.tgz.1
deportistaError.csv  juegos.csv  sample_data		spark-3.5.1-bin-hadoop3.tgz.2


In [ ]:
!pwd

/content




```
# Tiene formato de código
```

Aquí voy a instalar el paquete **Findspark** va a buscar la instalación de Spark y va configurar las variables de entorno necesarias para que PySpark pueda funcionar y luego tengo que mostrar la lista de los directorios para ver la ruta donde está Spark.

In [ ]:
!pip install -q findspark

In [ ]:
!ls /usr/lib/jvm

java-1.11.0-openjdk-amd64  java-11-openjdk-amd64  java-1.8.0-openjdk-amd64  java-8-openjdk-amd64


Me voy a quedar con el Java8, no sé porqué pero así se hizo el ejercicio. ❓Creo que fue el Java 8 el que se descargó en el ejercicio anterior. Y aquí se estba buscando su unicación para trarlo en el comando de abajo.

In [ ]:
!ls /usr/lib/jvm/java-8-openjdk-amd64/

ASSEMBLY_EXCEPTION  bin  docs  include	jre  lib  man  src.zip	THIRD_PARTY_README


###Segundo paso: **Ahora tengo que configurar las variables del entorno**

Previo a ello traje las herramientas de Java y Spark del ambiente Python
y luego se forzó a trabajar con java 8 en openjdk.amb64.



In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.1-bin-hadoop3"
!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
!java -version

openjdk version "1.8.0_402"
OpenJDK Runtime Environment (build 1.8.0_402-8u402-ga-2ubuntu1~22.04-b06)
OpenJDK 64-Bit Server VM (build 25.402-b06, mixed mode)


###Tercer paso: **Usar los RDD**

En los pasos anteriores configuramos Java, configuramos Spark Home y ya podemos usar los RDD

Para trabajar en colab con Python, tenemos que configurar las variables de enrtrno, vamos a usar las librerías que importamos hace rato.

In [ ]:
import findspark
findspark.init()

Ahora aquí se va a crear la ejecución dentro de Spark, para eso se van a usar las librerías de Spark; es decir, se importan. Luego se va a crear una variable llamada SC y se utiliza el objeto SparkContext y ese objeto va a recibir ciertos elementos, los más básicos son la creación de un ambiente con un nombre y dónde se va a ejecutar

In [ ]:
from pyspark import SparkContext

In [ ]:
sc.stop()

In [ ]:
sc = SparkContext(master="local", appName="Primer ejercicio Spark RRS")

In [ ]:
sc

<SparkContext master=local appName=Primer ejercicio Spark RRS>

Ahora sí vamos a trabajar con RDD, se va a crear diversas variables llamadas RDD 1 a la n, se van a usar paralela y distribuida, y como son nativos solo se neceista el Sparkcontext

In [ ]:
rdd1 = sc.parallelize([1,2,3,4,5])

In [ ]:
type(rdd1)

pyspark.rdd.RDD

In [ ]:
rdd1.collect()

[1, 2, 3, 4, 5]

###**Paso cuatro**

Ahora sí ya vamos a trabajar con los archivos de los deportes, porque ya está ejecutado el entorno en Spark lo primero que hago es conectar el dirve donde están los datos.

Ahora vamos a cargar a un RDD uno de los archivos, en este caso el de los países

In [ ]:
equiposOlimpicosRDD = sc.textFile("paises.csv")

In [ ]:
equiposOlimpicosRDD.collect()

Pero este collect me trae la información no distribuida por comas por eso mejor uso algunas funciones de Python

In [ ]:
equiposOlimpicosRDD = equiposOlimpicosRDD.map(lambda e: e.split(","))

In [ ]:
equiposOlimpicosRDD.collect()

In [ ]:
equiposOlimpicosRDD.take(5)

[['id', 'equipo', 'sigla'],
 ['1', '30. Februar', 'AUT'],
 ['2', 'A North American Team', 'MEX'],
 ['3', 'Acipactli', 'MEX'],
 ['4', 'Acturus', 'ARG']]

Para defiir qué tipo de variable es cada una, si de texto o de número lo que se puede hacer es trabajar con tuplas de tipo (key,value)

In [ ]:
equiposOlimpicosRDD.count()

1185

In [ ]:
equiposOlimpicosRDD = equiposOlimpicosRDD.map(lambda x: (x[2],x[1]))

In [ ]:
equiposOlimpicosRDD.take(5)

[('sigla', 'equipo'),
 ('AUT', '30. Februar'),
 ('MEX', 'A North American Team'),
 ('MEX', 'Acipactli'),
 ('ARG', 'Acturus')]

In [ ]:
equiposOlimpicosRDD.count()

1185

In [ ]:
equiposOlimpicosRDD.map(lambda ele: ele[0]).distinct().count()

231

In [ ]:
equiposOlimpicosRDD.take(5)

[('sigla', 'equipo'),
 ('AUT', '30. Februar'),
 ('MEX', 'A North American Team'),
 ('MEX', 'Acipactli'),
 ('ARG', 'Acturus')]

In [ ]:
#Filter conserva elementos que cumplen con la condición dada
encabezado = equiposOlimpicosRDD.first()
equiposOlimpicosRDD = equiposOlimpicosRDD.filter(lambda e: e != encabezado)

In [ ]:
equiposOlimpicosRDD.take(5)

[('AUT', '30. Februar'),
 ('MEX', 'A North American Team'),
 ('MEX', 'Acipactli'),
 ('ARG', 'Acturus'),
 ('AFG', 'Afghanistan')]

In [ ]:
#Aquí vamos a utilizar groupByKey
equiposOlimpicosRDD.map(lambda e: (e[0],e[1]) ).groupByKey().take(5)

[('AUT', <pyspark.resultiterable.ResultIterable at 0x7d3ab2dd3850>),
 ('MEX', <pyspark.resultiterable.ResultIterable at 0x7d3ab2dd3880>),
 ('ARG', <pyspark.resultiterable.ResultIterable at 0x7d3ab2dd38e0>),
 ('AFG', <pyspark.resultiterable.ResultIterable at 0x7d3ab2dd3940>),
 ('IRL', <pyspark.resultiterable.ResultIterable at 0x7d3ab2dd39a0>)]

In [ ]:
equiposOlimpicosRDD.map(lambda e: (e[0],e[1]) ).groupByKey().mapValues(list).take(5)

[('AUT',
  ['30. Februar',
   'Austria',
   'Austria-1',
   'Austria-2',
   'Breslau',
   'Brigantia',
   'Donar III',
   'Evita VI',
   'May-Be 1960',
   '"R.-V. Germania; Leitmeritz"',
   'Surprise']),
 ('MEX',
  ['A North American Team',
   'Acipactli',
   'Chamukina',
   'Mexico',
   'Mexico-1',
   'Mexico-2',
   'Nausikaa 4',
   'Tlaloc',
   'Xolotl']),
 ('ARG',
  ['Acturus',
   'Antares',
   'Arcturus',
   'Ardilla',
   'Argentina',
   'Argentina-1',
   'Argentina-2',
   'Blue Red',
   'Covunco III',
   'Cupidon III',
   'Djinn',
   'Gullvinge',
   'Matrero II',
   'Mizar',
   'Pampero',
   'Rampage',
   'Tango',
   'Wiking']),
 ('AFG', ['Afghanistan']),
 ('IRL',
  ['Akatonbo',
   'Aletta',
   'Ireland',
   'Ireland-1',
   'Ireland-2',
   'The Cloud',
   'Three Leaves'])]

In [ ]:
equiposOlimpicosRDD.map(lambda e: (e[0],e[1]) ).groupByKey().mapValues(len).take(5)

[('AUT', 11), ('MEX', 9), ('ARG', 18), ('AFG', 1), ('IRL', 7)]

In [ ]:
equiposOlimpicosRDD.countApprox(20)

1184

In [ ]:
deportista1RDD = sc.textFile("deportista.csv").map(lambda line: line.split(","))
deportista2RDD = sc.textFile("deportista2.csv").map(lambda line: line.split(","))

In [ ]:
deportista1RDD.take(5)

[['deportista_id', 'nombre', 'genero', 'edad', 'altura', 'peso', 'equipo_id'],
 ['1', 'A Dijiang', '1', '24', '180', '80', '199'],
 ['2', 'A Lamusi', '1', '23', '170', '60', '199'],
 ['3', 'Gunnar Nielsen Aaby', '1', '24', '0', '0', '273'],
 ['4', 'Edgar Lindenau Aabye', '1', '34', '0', '0', '278']]

In [ ]:
deportista2RDD.take(5)

[['67787', 'Lee BongJu', '1', '27', '167', '56', '970'],
 ['67788', 'Lee BuTi', '1', '23', '164', '54', '203'],
 ['67789', 'Anthony N. Buddy Lee', '1', '34', '172', '62', '1096'],
 ['67790', 'Alfred A. Butch Lee Porter', '1', '19', '186', '80', '825'],
 ['67791', 'Lee ByeongGu', '1', '22', '175', '68', '970']]

In [ ]:
deportistaOlimpicoRDD = deportista1RDD.union(deportista2RDD)

In [ ]:
deportistaOlimpicoRDD.take(5)

[['deportista_id', 'nombre', 'genero', 'edad', 'altura', 'peso', 'equipo_id'],
 ['1', 'A Dijiang', '1', '24', '180', '80', '199'],
 ['2', 'A Lamusi', '1', '23', '170', '60', '199'],
 ['3', 'Gunnar Nielsen Aaby', '1', '24', '0', '0', '273'],
 ['4', 'Edgar Lindenau Aabye', '1', '34', '0', '0', '278']]

In [ ]:
deportistaOlimpicoRDD.count()

135572

In [ ]:
deportista1RDD.count()

67787

In [ ]:
deportista2RDD.count()

67785

In [ ]:
sc.stop()